In [ ]:
import sys
import os
import torch
import torch.nn as nn
import torch.optim as optim
sys.path.append(os.path.abspath("D:\\burtm\\Visual_studio_code\\PD_related_projects"))

In [ ]:
selected_model = "trocr-small-stage1"
selected_transform = "trocr-small-stage1"
N_max=282
use_patches=True
pretrained=True
depth=2
hugging=True
num_epochs=10
batch_size=32
learning_rate=0.001
input_filename="icdar_train_df_patches_cc.csv"
criterion_name="CrossEntropyLoss"
criterion = training_utils.get_criterion(criterion_name)
optimizer_name = "Adam"
num_classes = 2  # Change this to match your dataset
early_stopping=10
scheduler_name = 'no_scheduling'#CosineAnnealingLR'
checkpoint_path = "D:\\burtm\Visual_studio_code\PD_related_projects\checkpoints\\"
models_path = "D:\\burtm\Visual_studio_code\PD_related_projects\outputs\models\\"

In [ ]:
# Example training metadata
training_metadata = {
    "type_of_approach": f"progressive fine tuning of {selected_model}",
    "type_of_approach_sigla": "pft",
    "model_name": selected_model,
    "transform_name": selected_transform,
    "epochs": num_epochs,
    "batch_size": batch_size,
    "learning_rate": learning_rate,
    "optimizer": optimizer_name,
    "pretrained": pretrained,
    "depth": depth,
    "use_patches": use_patches,
    "input_filename": input_filename,
    "num_classes": num_classes,
    "criterion_name": criterion_name,
    "early_stopping": early_stopping,
    "N_max": N_max,
    "scheduler_name": scheduler_name,
}

In [ ]:
transform=utils_transforms.get_transform(selected_transform,use_patches=use_patches)
train_dataloader,val_dataloader=data_loading.get_dataloaders(transform, batch_size=16, N_max=282, 
                                                file_name='icdar_train_df_patches_complete_cc.csv',huggingface=hugging)


In [ ]:
# Modify the final classification layer (assuming you have 10 classes)
num_classes = 2  # Change this to match your dataset

model=model_utils.get_model(selected_model, pretrained=pretrained, num_classes=num_classes)

# Define loss function and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is: ",device)
model = model.to(device)

In [ ]:
optimizer = training_utils.get_optimizer(model, optimizer_name, lr=learning_rate)
scheduler = training_utils.get_scheduler(optimizer, scheduler_name, T_max=num_epochs)

In [ ]:
def get_layer_groups(name):
    if name=='resnet18':
        layer_groups = [model.fc,model.layer4, model.layer3, model.layer2, model.layer1]  # Deepest to shallowest layers
    elif name=='trocr-small-stage1':
        layer_groups = [model.encoder.layer[11], model.encoder.layer[10], model.encoder.layer[9], 
                        model.encoder.layer[8], model.encoder.layer[7], model.encoder.layer[6], model.encoder.layer[5],
                        model.encoder.layer[4], model.encoder.layer[3], model.encoder.layer[2], model.encoder.layer[1],
                        model.encoder.layer[0]]
    else:
        raise ValueError(f"Unknown model name: {name}")
    return layer_groups



def progressive_unfreezing(model,model_name, train_loader,val_loader, 
                           criterion, device, base_lr=1e-3, num_epochs_per_stage=2, checkpoint_path=None, optimizer_name='Adam'):
    all_train_losses, all_val_losses = [], []
    # Freeze all layers except the classifier
    for param in model.parameters():
        param.requires_grad = False
    
    # Get the layer groups for progressive unfreezing
    layer_groups = get_layer_groups(model_name)

    # Unfreeze the classifier
    classifier_head=layer_groups[0]
    for param in classifier_head.parameters():
        param.requires_grad = True

    # Define optimizer for the classifier only
    optimizer = training_utils.get_optimizer(classifier_head.parameters(), optimizer_name, lr=base_lr)
    
    print('Step 1: training classification head')
    model, train_losses, val_losses = training_utils.train_model(model, train_loader, val_loader, criterion, 
                                                  optimizer, device, num_epochs=num_epochs_per_stage,
                                                  checkpoint_path=checkpoint_path+'\classifier.pth')
    all_train_losses.extend(train_losses)
    all_val_losses.extend(val_losses)
    print('model on:',next(model.parameters()).device)

    # **Step 2: Unfreeze layers progressively**
    print('Step 2: Unfreeze layers progressively')
    lr = base_lr * 0.1  # Reduce learning rate for deeper layers

    for i,layer in enumerate(layer_groups[1:]):
        # Unfreeze the current layer
        for param in layer.parameters():
            param.requires_grad = True  

        # Define new optimizer with unfrozen layers
        optimizer = training_utils.get_optimizer(filter(lambda p: p.requires_grad, model.parameters()), optimizer_name, lr=base_lr)

        # Train again with the newly unfrozen layer
        print(f"\nUnfreezing {layer} and training...")
        model, train_losses, val_losses = training_utils.train_model(model, train_loader, val_loader, criterion, 
                                                      optimizer, device, num_epochs=num_epochs_per_stage,
                                                      checkpoint_path=checkpoint_path+f'fine_tuning_layer_{i}.pth')
        all_train_losses.extend(train_losses)
        all_val_losses.extend(val_losses)
        print('model on:',next(model.parameters()).device)
        # Decrease learning rate for stability
        lr *= 0.1  
    print('fine tuning complete')
    return model, all_train_losses, all_val_losses

In [ ]:
model, train_losses,val_losses = progressive_unfreezing(model,selected_model, train_dataloader,val_dataloader, 
                                                        criterion, device, base_lr=1e-3, num_epochs_per_stage=2, 
                                                        checkpoint_path='D:\\burtm\\Visual_studio_code\\PD_related_projects\\checkpoints',
                                                        optimizer_name=optimizer_name)

# easy access

In [ ]:
def reload_modules():
    import importlib
    import utils.data_loading as data_loading
    import utils.visualization as visualization
    import utils.dataframes as dataframes
    import utils.utils_transforms as u_transforms
    import utils.training_utils as training_utils
    import utils.model_utils as model_utils
    

    importlib.reload(data_loading)
    importlib.reload(visualization)
    importlib.reload(dataframes)
    importlib.reload(u_transforms)
    importlib.reload(model_utils)
    importlib.reload(training_utils)

    return data_loading, visualization, dataframes, u_transforms, training_utils, model_utils
data_loading, visualization, dataframes, u_transforms, training_utils, model_utils = reload_modules()